# Monte Carlo Methods
Recall that there are two fundamental tasks - evaluating policies (aka the _prediction problem_: given a policy $\pi$, what is its value function $v_{\pi}$ or $q_{\pi}$?), and discovering optimal policies (aka the _control problem_).  Dynamic programming approaches all require knowledge of the dynamics $p(s',r|s,a)$, which is a tall order in many settings.  Monte Carlo methods are the first ones that only require _experience_ - from actual or simulated interaction with the target environment. A key advantage is that the dynamics are not required - rather, we only need to be able to generate reasonable transitions instead of marginalizing over the full distribution. 

For purposes of this chapter, we also restrict to episodic tasks, and to methods that use full episodes (i.e., full runs from an initial state to termination, so the returns are completely observed for the episode).  In some ways, this approach is like the bandit algorithms we looked at - except that we are estimating the expected total returns (i.e., discounted over subsequent states too), and for each state or state-action pair, separately.  Note that if we are also doing a search for optimal policies, this is inherently non-stationary. 




## Monte Carlo Prediction
Given a policy $\pi$, we want to estimate $v_{\pi}(s)$ for each $s \in \mathcal{S}$, given a set of episodes obtained by following $\pi$ and starting at state $s$.  Note that a given $s$ may be visited multiple times during an episode - how are these treated?  In _first-visit_ MC, we only use the first time a state is visited in each episode. This is analytically simple.  In _every-visit_ MC, we consider all visits to state s, but note that these are now no longer independent of each other within episodes.  In first-visit MC, everything is IID and our estimates are consistent and unbiased, with the standard deviation of the error falling as $1/\sqrt{n}$.  Every-visit MC is also consistent and converges quadratically but is harder to analyze.  Below is the algorithm for first-visit MC...  

```
First Visit MC for policy evaluation (prediction)

Input: policy pi
Initialize: 
    V[s] for all s in S             # The state value function
    Returns[s] = [] for all s in S  # Accumulate returns for each state

while True:
    Generate a length T episode following pi: [(S[0],A[0],R[1]), (S[1],A[1],R[2]), ..., (S[T-1],A[T-1],R[T])]
    G = 0
    For each step of the episode t = T-1, T-2, ..., 0 # Note that this is in reverse order!
        G = gamma * G + R[t+1]  
        if S[t] is in S[:t-1]:  # If doing every-visit, skip this check and just update Returns and V.
            continue
        else:
            Returns[S[t]].append(G)
            V[S[t]] = mean(Returns[S[t]])

```

An interesting point about MC - the cost of estimating the value of a given state doesn't really depend on the number of states (explicitly - in reality, it seems like it actually does since if there are lots of states, unless we can explicitly start in a given state, it may be hard to accumulate lots of relevant samples; additionally, the frequency of visiting a given state may depend on the current policy!). 

### Blackjack
This is an interesting case of a naturally episodic game in which it is easy to generate transitions but rather tedious and difficult to try to explicitly calculate the transition function...  This is the running example for this chapter. 

## Monte Carlo Estimation of Action Values
If no dynamics model is available, it can be particularly useful to focus on estimating the action values instead of the state values.  Why is this?  Consider how you would determine a policy given $p(s',r|s,a)$ and $v_{\pi}$: you could use the Bellman equation and simply do a one step lookahead over possible actions, evaluating the expected reward for each and the expected return from the next state since we have a dynamics model.  But if we don't have that, we can't do this procedure.  Whereas with $q_{\pi}(s,a)$, we could select actions by simply picking the argmax over actions of this function with a fixed $s$.  

The MC algorithm for estimating action values is essentially the same as for estimating state values, except that we are estimating for combinations of states and actions instead of just states.  We have a similar choice regarding first- or every-visit.  There is a new wrinkle however - what if $\pi$ simply doesn't pick some combinations of $s,a$?  Or it is deterministic as an extreme?  Then we run into a problem of never getting any samples for some $(s,a)$.  This problem is called _maintaining exploration_, and is analogous to the same issue in bandits, where we discussed $\epsilon-greedy$ and $UCB$ algorithms.  

One solution is to assume _exploring starts_ - basically, we ensure that episodes start with $(s,a)$ with non-zero probability for every $(s,a)$.  This is hard to justify unless we have a kick ass simulation, so instead we might consider policies that have non-zero probability for every $(s,a)$. Exploring starts is definitely problematic when we are actually interacting in a real environment as opposed to a simulation! 

## Monte Carlo Control
Recall that the _control problem_ is basically policy optimization - find a policy that maximizes total discounted returns.  The basic approach is to proceed as if we were doing dynamic programming, with some modifications.  In DP, we defined generalized policy iteration (GPI), which maintains an approximate value function along with an evolving policy.  GPI takes alternating steps of improving the value function approximation and improving the policy, with various degrees of granularity in each... 

Classical policy iteration does full steps of policy evaluation and policy improvement by greedifying the policy with respect the current value function (which is trivial if we are using $q_{\pi}(s,a)$ - we simply take $\pi(s) = \underset{a}{argmax} q_{\pi}(s,a)$.  The basic idea here is that the policy improvement theorem still holds - nothing has changed if we have infinite time and exploring starts. If we assume exploring starts, and don't care how long things take so we can do infinite episodes, then this will work and we'll converge to an optimal policy.  But this becomes pretty impractical if we are doing policy evaluation via MC. 

So to recap, there are two unrealistic assumptions in the above guarantee - first, that we have exploring starts; we have to figure out a way around this.  Second, we have to be able to do things with finite samples.  We tackle the latter issue first.  

Our basic approach is to relax the need for do full policy evaluation steps before policy improvement.  At an extreme, we might do _value iteration_, in which we only do a very limited evaluation (one sweep through states?) between each iteration of policy improvement.  Even more extreme - we could do it in-place, so we are only updating the action value function for one $(s,a)$ at a time.  Generally, in MC, we alternate between evaluation and improvement on an episode by episode basis: After each episode (generated by following the current policy), we update the action value function for the $(s,a)$ observed in the episode, and then we do a round of policy improvement.  Note that this assumes exploring starts...  

```
Initialize:
pi[s] for all s
Q[s,a] for all s, a
Returns[s,a] = [] for each s, a

while True: 
    Choose s0 from S, a from A(s0) randomly, with non-zero for all (s,a) <- exploring starts
    # Each episode is a sequence of tuples (S[t], A[t], R[t+1]).  
    Generate an episode of length T-1 by following pi. 
    G = 0
    For each time step t = T-1, T-2, ..., 0:
        G = gamma * G + R[t+1] 
        if (S[t], A[t]) is in E[:t]: 
            continue # Not first visit
        else:
            Returns[S[t],A[t]].append(G)
            Q[S[t],A[t]] = mean(Returns[S[t],A[t]])
            pi[S[t]] = argmax over a of Q[S[t],a]
```

Note that in MC ES, all returns for each $(s,a)$ are averaged regardless of which policy gave rise to it.  But we can argue that this won't converge to a suboptimal policy - if it did, then it would converge to the action value function for that policy, and we would improve on it.  So we can argue that stability will only happen if the policy (and action value function) are optimal.  This has NOT, however, been formally proved - it's just an intuition.  

## MC Control without exploring starts - On Policy
There are two approaches to avoiding exploring starts.  Fundamentally, we need to keep selecting all $(s,a)$ with non-zero probability.  We can either evaluate and improve a policy that is constrained to meet this requirement directly - _on policy_ - or we can generate samples from a policy that meets this requirement and evaluate/improve another policy by reweighting observations via importance sampling.  In this section, we focus on the on-policy MC.  

The main idea is to restrict ourselves to _soft_ policies, in which $\pi(a|s) > 0 \forall a \in \mathcal{A}, s \in \mathcal{S}$.  Over time, we move towards deterministic optimal policies.  We take an approach that basically uses the $\epsilon$ greedy algorithm we used for bandits - with probability $1 - \epsilon$ we take the greedy action relative to the current action value function, and with probability $\epsilon$ we take an action uniformly at random. This sort of policy is a kind of $\epsilon$ _soft_ policy, in which each $\pi(a|s) \ge \frac{\epsilon}{|\mathcal{A}|}$.  The key is that for an $\epsilon$-soft policy $\pi$, an $\epsilon$-greedy policy with respect to $q_{\pi}$ will be better than or equal to $\pi$ (the policy improvement theorem holds).  Here is an algorithm: 

```
Iniitialize: 
pi is an epsilon soft policy
Q[s,a] for all s, a
Returns[s,a] = [] for each s,a

while True:
    # Note we don't do exploring starts!
    Generate an episode of length T, following pi
    G = 0
    for t = T-1, T-2, ..., 0:
        G = gamma * G + R[t+1]
        if S[t], A[t] in E[:t]: 
            # Not first visit to skip
            continue
        else:
            Returns[S[t], A[t]].append(G)
            Q[S[t], A[t]] = mean(Returns[S[t], A[t]])  # Update action-value function
            A* = argmax over a of Q[S[t],a]
            for all a: 
                if a == A*:
                    pi[a|s] = 1 - epsilon + epsilon/num_actions
                else:
                    pi[a|s] = epsilon / num_actions
```

We can prove the policy improvement theorem holds as stated above.  Let $\pi'$ be the $\epsilon$-greedy policy relative to $\epsilon$-soft policy $\pi$.  For any $s$, 

$$
\begin{align}
q_{\pi}(s, \pi'(s)) & = \sum_a \pi'(a|s)q_{\pi}(s,a) \\
  & = \frac{\epsilon}{|\mathcal{A}|} \sum_a q_{\pi}(s,a) + (1 - \epsilon) \max_a q_{\pi}(s,a) \\
  & \ge \frac{\epsilon}{|\mathcal{A}|} \sum_a q_{\pi}(s,a) + (1 - \epsilon) \sum_a \frac{\pi(a|s) - \frac{\epsilon}{|\mathcal{A}|}}{1-\epsilon} q_{\pi}(s,a) \\
  & = \frac{\epsilon}{|\mathcal{A}|} \sum_a q_{\pi}(s,a) + \sum_a \pi(a|s) q_{\pi}(s,a) - \frac{\epsilon}{|\mathcal{A}|} \sum_a q_{\pi}(s,a) \\
  & = v_{\pi}(s)
\end{align}
$$

In the third line, the inequality holds because the sum is a weighted average with weights summing to 1, so the sum is less than or equal to the largest number being averaged; there is nothing very mysterious about the weights - they sum to 1 when summed over $a$.  The second line is just the definition of an $\epsilon$-greedy policy with respect to $\pi$.  Thus, $v_{\pi'}(s) \ge v_{\pi}(s) \forall s \in \mathcal{S}$

There is a longish, somewhat tortured proof sketch that equality holds only when $\pi'$ and $\pi$ are both optimal among all $\epsilon$-soft policies but I lost patience for it.  The basic idea is to imagine a new environment in which, with probability $\epsilon$, whatever action is taken, the environment picks a new action randomly; otherwise the environment behaves exactly the same as before.  The key is that the best you can do in the new environment with _any_ policy is the same as you could do in the old environment with an $\epsilon$-soft environment. So we can now do away with exploring starts, but we have the optimal policy among $\epsilon$-soft policies instead of all policies...

## Off-policy Prediction via Importance Sampling
Learning control methods (not DP) have to balance optimizing a policy (moving a policy in a greedy fashion) but also exploring enough to actually find the optimal policy.  The on-policy approach above compromises by finding an optimal policy that is optimal from a set of policies that explore; it could be that for some bizarre reason the optimal policies from this set are very far from optimal, though I bet that doesn't happen that often in practice.  

We can also try a different approach - we maintain two policies.  The first, the _behavior_ policy, is what is used to decide on actions, and it is exploratory.  The second is the _target policy_ - that is the one we are optimizing.  Because we are optimizing using data from a different policy, i.e., from data that is "off" the target policy, this is called _off-policy learning_.  Note that this is a different situation from above, where we used episodes from different policies to estimate action values.  This is much more severe in the sense that the behaviors are always generated from a fixed, different policy.  In value iteration, the most recent samples are at least closer to the current policy! 

In this section, we consider the off-policy prediction problem - that is, the target policy $\pi$ and the behavior policy $b$ are fixed.  How can we estimate the value function for the target policy given episodes generated by the behavior policy?  

First, we must require that every action under $\pi$ is also taken under $b$, that is, that there is support, so $\pi(a|s) > 0 \rightarrow b(a|s) > 0$. In RL, this is called _coverage_.  Note that $b$ can take actions that $\pi$ never takes!  In particular, note that $\pi$ can be deterministic, as might be the case if we are constructing the deterministic greedy policy with respect to the value function.  But for now, assume $\pi$ is fixed.  

As you might guess, the key idea is to use _importance sampling_ to estimate expected values under $\pi$ from samples from $b$.  Let's review this quickly.  The basic idea is - we want to estimate $\mathbb{E}_{p}[X]$ from a sample from another distribution $b(X)$ - ${X_1, X_2, \dots, X_N} \sim b(X)$, where the sample is i.i.d.  
$$
\begin{align}
\mathbb{E}_p[X] & = \int p(X) X dX \\
& = \int X \frac{p(X) b(X)}{b(X)} X dX \\
& = \int b(X) \rho(X) X dX \\
& = \mathbb{E}_b[\rho(X) X] \\
& \approx \frac{1}{N} \sum_{i=1}^N \rho(X_i) X_i \text{ where } X_i \sim b(X), \rho(X) = \frac{p(X)}{b(X)}
\end{align}
$$

So we estimate the expectation under $p$ by using a sample under $b$, with each sample weighted by the ratio of the probabilities under $p$ and $b$.  


How do we apply this idea to trajectories, etc?  Say we have a trajectory $\Theta = S_t, A_t, S_{t+1}, A_{t+1}, S_{t+2}, \dots, A_{T-1}, S_T$. Under policy $\pi$, the probability of this sequence is: 

$$
\begin{align}
Pr_{\pi}(\Theta) & = \pi(A_t|S_t) p(S_{t+1}|S_t,A_t) \pi(A_{t+1}|S_{t+1}) p(S_{t+2}|S_{t+1},A_{t+1}) \dots p(S_T|S_{T-1},A_{T-1}) \\
& = \prod_{k=t}^{T-1} \pi(A_k|S_k) p(S_{k+1}|S_k,A_k)
\end{align}
$$

Similarly, under policy $b$, it is just: 

$$
\begin{align}
Pr_b(\Theta) & =  \prod_{k=t}^{T-1} b(A_k|S_k) p(S_{k+1}|S_k,A_k)
\end{align}
$$

Note that both of these expressions still require the state transition function, and we are assuming we don't have access to the dynamics function!  But that's not a problem - when we form the ratio $\rho(\Theta)$ we want to use for importance sampling, we get: 
$$
\begin{align}
\rho_{t:T-1} & = \frac{Pr_{\pi}(\Theta)}{Pr_{b}(\Theta)} \\
& = \prod_{k=t}^{T-1} \frac{\pi(A_k|S_k)}{b(A_k|S_k)}
\end{align}
$$

i.e., the transition part cancels out, and we presumably can calculate the two probabilities (this isn't always a given - sometimes things are represented in ways that are easy to generate samples from but are not so easy to use to calculate probabilities!). 

So - given a trajectory generated under $b$, and a target policy $\pi$, and the cumulative discounted return $G_t$, we note that: 

$v_b(s) = \mathbb{E}_b[G_t|S_t = s] \neq \mathbb{E}_{\pi}[G_t|S_t = s]$.

But: 

$$
\begin{align}
v_{\pi}(s) & = \mathbb{E}_{\pi}[G_t|S_t = t] \\
& = \mathbb{E}_b[\rho_{t:T-1} G_t | S_t = t ]
\end{align}
$$


Next, they introduce some notational sugar to make things easier.  The idea is to number time steps such that they are sequential across episode boundaries.  Assume some order (arbitrary) of episodes, and time steps that are sequential within and across this ordering of episodes.  Let $T(t)$ be the first time of termination (episode end) after time $t$, and $G_t$ the return after $t$ up to the end of the episode at $T(t)$.  Finally, let $\mathcal{T}(S)$ be the set of times $t$ in which state $s$ is visited (or if doing first-visits, then the first such time in each episode).  Then the set ${G_t}_{t \in \mathcal{T}(s)}$ are the returns that are relevant for estimating the value of state $s$, and ${\rho_{t:T(t)-1}_{t \in \mathcal{T}(s)}$ are the importance sampling weights, so: 

$$
V_{\pi}(s) = \frac{\sum_{t \in \mathcal{T}(s)} \rho_{t:T(t)-1} G_t}{|\mathcal{T}(s)|}
$$

This is called _ordinary importance sampling_.  An alternative is _weighed importance sampling_: 

$$
V_{\pi}(s) = \frac{\sum_{t \in \mathcal{T}(s)} \rho_{t:T(t)-1} G_t}{\sum_{t \in \mathcal{T}(s)} \rho_{t:T(t)-1}}
$$

What's the difference between these two options?  Consider what happens under first-visit, whe there is exactly one return for state $s$ observed.  In this case, the ordinary IS estimate is still an unbiased estimator for $v_{\pi}(s)$, but it may be very high variance - consider what happens if the $\rho$ is 10.  In that case the estimate is 10x off (even though it is statistically unbiased).  What about the weighted IS estimate?  In that case, the weights simply cancel out in the numerator and denominator, and the estimate is simply $G_t$.  But that is from policy $b$, so it is biased...  

More generally, ordinary IS is unbiased but potentially high variance if the behavior policy is quite different from the target policy (because the $\rho$ can be very large).  It may help to re-name things in more familiar terms - the behavior policy is also called the proposal distribution in other contexts.  Weight IS, on the other hand, is biased, but much lower variance (because the $\rho$'s are bounded).  

In practice, the weighted estimator is used more because of it's much lower variance. 

For every-visit, both estimators are biased, though the bias decays to 0 asymptotically as sample size increases.  In practice, every visit methods are used because they are simpler to write up (we don't have to check for first visits). 

## Incremental updates
As before we can make things more efficient by using incremental updates as we did for bandits.  In fact, for on-policy MC methods the translation is trivial.  For off-policy MC, we have to treat ordinary and weighted IS separately.  For ordinary IS, we just weight each $G_t$ by $\rho_{t:T-1}$.  Things get a little bit more complicated for weighted IS however, but it's still pretty straight forward: 

Let $C_n = \sum_{k=1}^{n} W_k$, where the $W_k$ are the IS weights.  

$$
\begin{align}
V_{n+1} & = \frac{\sum_{k=1}^{n} W_k G_k}{C_n} \\
& = \frac{W_n G_n}{C_n} + \frac{1}{C_n} \sum_{k=1}^{n-1} W_k G_k \\
& = \frac{W_n G_n}{C_n} +  \frac{C_{n-1}}{C_n C_{n-1}} \sum_{k=1}^{n-1} W_k G_k \\
& = \frac{W_n G_n}{C_n} +  \frac{C_{n-1}}{C_n} \frac{1}{C_{n-1}} \sum_{k=1}^{n-1} W_k G_k \\
& =  \frac{W_n G_n}{C_n} + \frac{C_n - W_n}{C_n} V_n \\
& = V_n + \frac{W_n}{C_n} (G_n - V_n)
\end{align}
$$

The algorithm for off-policy MC prediction with weighted IS is: 

```
Inputs: 
Target policy pi[s,a]
Proposal or behavior policy b[s, a]

Initialize: 
Q[s,a] for all s, a
C[s, a] for all s, a   # Cumulative weights

while True:
    Generate an episode of length T by following
    G = 0
    W = 1
    For each t = T-1,T-2,...,0: 
        if W == 0: break
        G = gamma * G + R[t+1]
        C[S[t], A[t]] += W
        Q[S[t], A[t]] += W * (G - Q[S[t],A[t]]) / C[S[t],A[t]]
        W = W * pi[S[t], A[t]] / b[S[t], A[t]]
```

<b>Exercise 5.9</b> First visit on-policy MC prediction: 
```
First Visit MC for policy evaluation (prediction), cumulative updates version

Input: policy pi, alpha             # alpha is step size
Initialize: 
    V[s] for all s in S             # The state value function

while True:
    Generate a length T episode following pi: [(S[0],A[0],R[1]), (S[1],A[1],R[2]), ..., (S[T-1],A[T-1],R[T])]
    G = 0
    For each step of the episode t = T-1, T-2, ..., 0 # Note that this is in reverse order!
        G = gamma * G + R[t+1]  
        if S[t] is in S[:t-1]:  # If doing every-visit, skip this check and just update Returns and V.
            continue
        else:
            V[S[t]] = V[S[t]] + alpha * (G - V[S[t]])
```                    

<b>Exercise 5.10</b> See above.

## Off-policy MC control
Here is an algorithm for off-policy MC control, with weighted importance sampling and a deterministic target policy: 

```
Initialize: 
Q[s,a] 
C[s,a]
pi[s] = argmax over a of Q[s,a] for each s

while True:
    Generate a length T episode by following b
    G = 0
    W = 1
    for t = T-1,T-2,...,1,0:
        G = gamma*G + R[t+1]
        C[S[t],A[t]] += W
        Q[S[t],A[t]] += W * (G - Q[S[t],A[t]]) / C[S[t],A[t]]
        W = W / b[A[t],S[t]]
        pi[S[t]] = argmax over a of Q[S[t], a]  # Just deal with ties appropriately... 
        if A[t] != pi[S[t]]: 
            break  # Exit the inner loop - WHY?!  
```